In [107]:
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

## Getting page content by Selenium

We need to load html content by selenium to avoid the Beautiful Soup load until the entire webpage is loaded.

In [289]:
URL = "https://velacat.sailti.com/ca/default/races/resultsajax/id/5047/idsc2r/15119/allResults/1/handicap"

In [290]:
COMPETICAO = "World Championship 2021"
CATEGORIA = "ILCA 7"

In [291]:
options = Options()
options.add_argument("--headless")

with Chrome(options=options) as browser:
    browser.get(URL)
    html = browser.page_source

In [292]:
soup = BeautifulSoup(html, "html.parser")

In [293]:
def get_flotilha(soup_object):
    """
    This functions gets a input like <span style="display:none;">punt_Yellow _0000003</span>
    and returns the series of this data.
    """
    if "Yellow" in str(soup_object) or "Blue" in str(soup_object):
        return "G"
    elif "Silver" in str(soup_object):
        return "P"
    elif "Gold" in str(soup_object):
        return "O"
    elif "Medal" in str(soup_object):
        return "Medal Race"
    else:
        return "TO VERIFY"
    

In [294]:
def get_pontuacao(text):
    """
    This functions geta a string like "'\n\npunt_Gold _0000002 2\n                                            \n'"
    and extract the part referent to the pontuation
    """
    try:
        pontuacao = text.split("\n")[2].split(" ")[-1]
    except:
        pontuacao= "TO VERIFY"
    return pontuacao
    

In [295]:
def is_data(css_class):
    return css_class=="odd" or css_class=="even"

In [305]:
data = {
    "Id Resultado": [],
    "Id Competidor": [],
    "Nome Competidor": [],
    "Id Competição": [],
    "Classe Vela": [],
    "Pontuação Regata": [],
    "Flotilha": [],
    "Posição Geral": [],
    "Punição": [],
    "Pontuação Total": [],
    "Nett": [],
    "Nome Competição": []    
}

for row in soup.find_all(class_ = is_data):

    for j in row.findChildren("td"):
        str_obj = str(j)
        if "position" in str_obj:
            position = j.text

        elif "netpoints" in str_obj:
            net = j.text

        elif "totalpoints" in str_obj:
            totalpoints = j.text

        elif "br" in str_obj and "class" not in str_obj:
            name = j.text   
 

    for race in row.find_all(attrs={"rel": True}):
        flotilha = get_flotilha(race)
        pontuacao = get_pontuacao(race.text)  
        data["Id Resultado"].append(None)
        data["Id Competidor"].append(None),
        data["Nome Competidor"].append(name),
        data["Id Competição"].append(None),
        data["Classe Vela"].append(CATEGORIA),
        data["Pontuação Regata"].append(pontuacao),
        data["Posição Geral"].append(position),
        data["Flotilha"].append(flotilha)
        data["Punição"].append(None),
        data["Pontuação Total"].append(totalpoints),
        data["Nett"].append(net),
        data["Nome Competição"].append(COMPETICAO) 


In [306]:
df= pd.DataFrame(data)
df.to_csv(f"results/{COMPETICAO+CATEGORIA}.csv")